In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like thi

In [3]:
#standard class syntax (Not pydantic)
# Doesn't do type checking
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email
        
foo = User(name="Joe",age="bar", email="joe@gmail.com")     
print(f"Joe's age is {foo.age}")  
foo
# we defined age as an INT, but it accepted a string happily.  WTH.

Joe's age is bar


In [4]:
# Pydantic allows you to strongly type your fields
class pUser(BaseModel):
    name: str
    age: int
    email: str
    
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")
foo_p #pydantic objects also print with useful information.

pUser(name='Jane', age=32, email='jane@gmail.com')

In [5]:
# This throws a validation error, because "bar" is not an INT.
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [7]:
# you can even next pydantic objects.  NOTE: a list of pUsers (defined above) is the data type for students.
class Class(BaseModel):
    students: List[pUser]

obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [8]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

# this class is logically the same as the function that we defined in previous lessons
# NOTE: the doc string is mandatory
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")
    
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [9]:
# if we don't include the doc string, convert_pydantic_to_openai_function throws an error
class WeatherSearch_NoDocString(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")
    
convert_pydantic_to_openai_function(WeatherSearch_NoDocString)

KeyError: 'description'

## Combining OpenAI functions with LCEL

In [10]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI()
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [11]:
# of course, you can also bind the function, so you can use the function in the middle of a chain
model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

### Forcing it to use a function

In [12]:
# NOTE: the "fucntion_call" parameter identifying our function name
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})
model_with_forced_function.invoke("hi!") #not passing a weather related prompt, it still should return a response from WeatherSearch becuase it was forced.

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "JFK"\n}'}})

### Using in a chain

In [13]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])
chain = prompt | model_with_function
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

### Using Mutliple functions
Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [14]:
# defining a pydantic class
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")
    
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

model_with_functions = model.bind(functions=functions)

In [15]:
# should use the weather function, if we ask it a weather question
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [21]:
# should use the music function, if we ask it about somethign with songs
model_with_functions.invoke("Does prince sing on his emancipated album?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{\n  "artist_name": "Prince",\n  "n": 10\n}'}})

In [17]:
# doesn't use a function if neither topic is included in the prompt
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')